In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Widya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Data for SVM**

This following code prepared the dataset for machine learning. 

It creates the dataset with label ABU and NOT

In [17]:
df = pd.read_csv( "../data/ALYT_master.csv")

## drop unnecessary column and drop label 4 (uncertain) ##
df = df.drop(["comment_id", "label_normalised", "video_id", "video_topic", "video_type"], axis=1)
df = df[df['label'] != 4]

## map class number to abuse/no abuse ##
conditions = [
    (df['label'] >= 1) & (df['label'] <= 3),
    (df['label'] >= 5) & (df['label'] <= 7)
]

choices = ['NOT', 'ABU']

# Create a new column 'label2' based on the conditions, keeping the original 'label' where conditions are not met
df['label_class'] = np.select(conditions, choices, default=df['label'])
df.to_csv("../data/alyt.csv", index=False)

In [22]:
def clean_comment(comment): 
    """
    This function is used to clean the sentence froom column "comment" dataframe alyt
    :type comment: string
    """
    comment = re.sub(r"@[A-Za-z0-9]+", '', comment)
    comment = re.sub("#", "", comment)
    comment = re.sub(r"[^a-zA-Z\s]", "", comment) #Removes also inner *, but also emoticones
    comment = comment.lower()
    
#     # Remove stopwords
#     stop_words = set(stopwords.words('english'))  
#     words = tweet.split()
#     words = [word for word in words if word not in stop_words]
    
#     tweet = ' '.join(words)
    return comment

df_alyt = pd.read_csv("../data/alyt.csv")
df_alyt['comment'] = df_alyt['comment'].apply(clean_comment)
index_to_drop = 11 #has no sentence (I- ... I- ..... I- .......????????????????????!)
df_alyt = df_alyt.drop(index_to_drop)
df_alyt.to_csv("../data/alyt.csv", index=False)

In [24]:
## splitting the dataset into train and test. Ratio 70:30 ##
df_alyt = pd.read_csv("../data/alyt.csv")

X = df_alyt ["comment"]
y = df_alyt["label_class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y, random_state = 42) 

## save train data ##
data_train = {
    "comment" : X_train,
    "label" : y_train}

df = pd.DataFrame(data_train)
df.to_csv("../data/ALYT_train.csv", index = False)

## save test data ##
data_test = {
    "comment" : X_test,
    "label" : y_test}
df = pd.DataFrame(data_test)
df.to_csv("../data/ALYT_test.csv", index = False)

In [26]:
train = pd.read_csv("../data/ALYT_train.csv")
test = pd.read_csv("../data/ALYT_test.csv")
print(train.label.value_counts())
print()
print(test.label.value_counts())

label
NOT    11983
ABU     1592
Name: count, dtype: int64

label
NOT    5137
ABU     682
Name: count, dtype: int64


**Data for BERT**

This following codes prepare data for BERT 

it creates dataset with binary label (1 and 0)

In [33]:
bert_train = train
bert_train['label'] = bert_train['label'].replace({'NOT': 0, 'ABU': 1})
bert_train.to_csv('../data/ALYT_train_bert.csv', index=False)

In [34]:
bert_test = test
bert_test['label'] = bert_test['label'].replace({'NOT': 0, 'ABU': 1})
bert_test.to_csv('../data/ALYT_test_bert.csv', index=False)